In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("../input/snu-2021-1-ds-project-2/train.csv")
test = pd.read_csv("../input/snu-2021-1-ds-project-2/test.csv")


In [ ]:
predicted_train = train[['id', 's1', 'label']].copy() 
predicted_train['predicted_label'] = X['label']
predicted_train.head()

In [ ]:
X['label'] = 0
X.head()

In [ ]:
X.iloc[0, 2]

In [ ]:
X = pd.read_csv("../input/competition2/preprocessed_train.csv")
X['label'] = 0

# 2. 올라가는 추세이면 구간 확장하기
# 0, 1000, 2000, ... 마지막까지
start = 0 
end = 1000
temp_end = 1000
increasing = 1
prev_s1 = X.iloc[0, 2]
peak = prev_s1
trough = prev_s1
starting_val = X.iloc[0,2]
for i in range(0, X.index[-1]+1, 1000):
    next_s1 = X.iloc[i, 2]
    # 이전 구간이 올라가는 구간일 경우 
    if (increasing):
        # s1_mean이 올라가면 end를 업데이트해 구간 확장하기
        if (next_s1 >= prev_s1):
            if(peak < next_s1):
                peak = next_s1 
            
            temp_end = i +1000
            end = temp_end #값이 떨어지다 2000 이상에서 다시 올랐을 경우 실제 구간을 임시 구간으로 맞춰주기
        
        
        elif (next_s1 >= 2000):
            # 값이 떨어졌더라도 2000이상이면 우선 임시 구간 확장
            temp_end = i + 1000
            
        else:
            # 2000보다 떨어졌을 때 end까지의 구간 내의 label 값 설정하기
        
            predicted_label = (peak-1500)/5 + (peak-2500)*0.1
            if (peak < 2400):
                predicted_label = 0
            elif (peak - starting_val < 400):
                predicted_label = 0
            X.iloc[start:end, 3] = predicted_label

            increasing = 0
            start = end
            end = i + 1000
            trough = next_s1

    # 이전 구간이 내려가는 구간일 경우 
    else:
        # s1_mean이 올라가면 내려가는 구간을 끝내고 올라가는 구간 시작하기
        if (next_s1 >= prev_s1):
            # 내려가는 구간 내의 label 값 설정하기 
#             predicted_label = (trough-1500)/5 + (trough-2500)*0.1
#             if (predicted_label < 0):
#                 predicted_label = 0
            predicted_label = 0
            X.iloc[start:end, 3] = predicted_label 
            
            increasing = 1
            start = i 
            end = i + 1000
            peak = next_s1 
        # s1_mean이 내려가면 end를 업데이트해 구간 확장하기 
        else:
            trough = next_s1 
            end = i + 1000

    prev_s1 = next_s1 

# X['id'] = X.index 
# X = X[['id', 'label']].copy() 

In [ ]:
X.head()

In [ ]:
# plot의 사이즈를 지정하는 코드로  원하는 사이즈로 설정하면 됩니다.
plt.rcParams["figure.figsize"] = (18,5) 

def plot_time(df, time=(0, 100), columns='all'):
    df_time = df[(df['time']>=time[0]) & (df['time']<time[1])]
    column_list = df.columns if columns=='all' else columns

    for col in column_list:
        if col !='time':
            plt.plot('time', col, data=df_time)

    plt.legend(loc='center left', bbox_to_anchor=(1.02, 0.5))
    plt.show()

In [ ]:
predicted_train['time'] = predicted_train.index/100
plot_time(predicted_train, time=(0, 11000), columns=['s1', 'label', 'predicted_label'])

In [ ]:
sub_X = predicted_train[['s1', 'predicted_label']]
y = predicted_train['label']

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor   
from lightgbm import LGBMRegressor 

# model = DecisionTreeRegressor()
# model = RandomForestRegressor(n_estimators=10, random_state=0)
# model = GradientBoostingRegressor() 
model = AdaBoostRegressor() 
# model = LinearRegression() 
# model = Ridge() 
# model = Lasso() 
# model = KNeighborsRegressor() 
# model = LGBMRegressor(n_estimators=1000, learning_rate=0.01)  
# model.fit(sub_X, y)

model_name = type(model).__name__
 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error 

X_train, X_valid, y_train, y_valid = train_test_split(sub_X, y, train_size=0.8, test_size=0.2, random_state=0)

import time 

start_time = time.time() 
model.fit(X_train, y_train)
training_time = int(time.time() - start_time) 
print("training time: %s seconds" % training_time)

valid_preds = model.predict(X_valid)
MAE = mean_absolute_error(y_valid, valid_preds)
print('MAE: {}'.format(MAE))

In [ ]:
MAE = mean_absolute_error(predicted_train['label'], predicted_train['predicted_label'])
print('MAE: {}'.format(MAE))